In [32]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.callbacks import TensorBoard
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from multiprocessing import  Pool
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from time import time

In [5]:
#read train dataset
print("open dataset")
train = pd.read_csv("../dataset/goodreads_train.csv")

open dataset


In [6]:
# Load a np_archive of review_text col of train dataset preprocess in main.py
train_prepro = pd.DataFrame(data=np.load(file="../vocabulaires/prepro_train_archive_PN_less.npy", allow_pickle=True), columns=['review_text'])['review_text']


In [7]:
# add review text col to train dataset
train['review_text'] = train_prepro

In [8]:
print("0  ",train[train['rating'] == 0]["rating"].count())
print("1  ", train[train['rating'] == 1]["rating"].count())
print("2  ", train[train['rating'] == 2]["rating"].count())
print("3  ", train[train['rating'] == 3]["rating"].count())
print("4  ", train[train['rating'] == 4]["rating"].count())
print("5  ", train[train['rating'] == 5]["rating"].count())
#train_3_5 = train[train['rating'] >= 3]
#train['rating'] = train['rating'].apply(lambda x: x-3)

0   30988
1   28718
2   72627
3   188972
4   313688
5   265007


In [11]:
rating = keras.utils.to_categorical(train['rating'], num_classes=6)

In [12]:
#create model layers
inputs = keras.Input(shape=(1,), dtype=tf.string) # text
inputs2 = keras.Input(shape=(1), dtype=tf.int64) # n_comment
inputs3 = keras.Input(shape=(1), dtype=tf.int64) # n_votes
#create vectorize layer, to transform words in integer
vectorize_layer = keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    output_mode='int',
    output_sequence_length=1400,
    vocabulary=np.load('../vocabulaires/voc_lemm_without_NP.npy')
)(inputs)
conc = keras.layers.concatenate([vectorize_layer, inputs2,inputs3])
outputs = keras.layers.Dense(6, activation=tf.keras.activations.tanh)(conc)

In [13]:
model = keras.Model(inputs=[inputs, inputs2, inputs3], outputs=outputs, name="mnist_model")
tensorboard = TensorBoard(log_dir="../logs/relu".format(time()))

In [14]:
#learning_rates = [0.01, 0.001, 0.000001, 0.0000001]

In [15]:
#for learning_rate in learning_rates:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy]
              )
model.fit([train['review_text'], train['n_comments'], train['n_votes']], rating, epochs=5,
                  callbacks=[
                      tf.keras.callbacks.TensorBoard(log_dir="../logs/relu"),
                  ],
                  batch_size=100000
                  )

Epoch 1/5
9/9 [==============================] - 25s 3s/step - loss: 6.3555 - categorical_accuracy: 0.0913
Epoch 2/5
9/9 [==============================] - 24s 3s/step - loss: 4.7275 - categorical_accuracy: 0.1183
Epoch 3/5
9/9 [==============================] - 23s 3s/step - loss: 3.7915 - categorical_accuracy: 0.1149
Epoch 4/5
9/9 [==============================] - 23s 3s/step - loss: 3.3351 - categorical_accuracy: 0.0997
Epoch 5/5
9/9 [==============================] - 23s 3s/step - loss: 3.0892 - categorical_accuracy: 0.0874


In [16]:
#model.save("../models_trained/linear_model_2")

In [17]:
#read test dataset
test = pd.read_csv("../dataset/goodreads_test.csv")

In [25]:
# Load a np_archive of review_text col of train dataset preprocess in main.py
test_prepro = pd.DataFrame(data=np.load(file="../vocabulaires/prepro_test_archive_PN_less.npy", allow_pickle=True), columns=['review_text'])['review_text']
test['review_text'] = test_prepro

In [26]:
test

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,b9450d1c1f97f891c392b1105959b56e,7092507,5c4df7e70e9b438c761f07a4620ccb7c,* spoiler alert this definitely one favorite a...,Sat Nov 10 06:06:13 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sat Nov 10 00:00:00 -0800 2012,1,0
1,b9450d1c1f97f891c392b1105959b56e,5576654,8eaeaf13213eeb16ad879a2a2591bbe5,* spoiler alert `` you drink . '' i 'm huge fa...,Fri Nov 09 21:55:16 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,1,0
2,b9450d1c1f97f891c392b1105959b56e,15754052,dce649b733c153ba5363a0413cac988f,one favorite character under i 'm happy read s...,Fri Nov 09 00:25:50 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,0,0
3,b9450d1c1f97f891c392b1105959b56e,17020,8a46df0bb997269d6834f9437a4b0a77,* spoiler alert if feel like travelling n't mo...,Thu Nov 01 00:28:39 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Thu Nov 01 00:00:00 -0700 2012,0,0
4,b9450d1c1f97f891c392b1105959b56e,12551082,d11d3091e22f1cf3cb865598de197599,3.5 star i read enjoyed first two novel series...,Thu Oct 18 00:57:00 -0700 2012,Mon Apr 01 23:00:51 -0700 2013,Sat Mar 30 00:00:00 -0700 2013,Fri Mar 29 00:00:00 -0700 2013,0,0
...,...,...,...,...,...,...,...,...,...,...
478028,35cef391b171b4fca45771e508028212,15745950,0e1db3d4b04256f9660f5d276ddf1314,n't wait ' before ... after ...,Sun Aug 05 10:26:12 -0700 2012,Tue Apr 16 17:24:00 -0700 2013,Tue Apr 16 00:00:00 -0700 2013,NaN,0,0
478029,35cef391b171b4fca45771e508028212,10861195,0b7f352e58caf0fd1f961e98ef04e89c,to-read shelf forever . update i 've finished ...,Tue Jul 10 23:31:00 -0700 2012,Fri Dec 28 20:05:51 -0800 2012,NaN,NaN,0,0
478030,35cef391b171b4fca45771e508028212,6131164,9b19eff33ddb14e9e68fca2e90379e46,the last book left wanting . i need happy endi...,Tue Jul 10 19:45:17 -0700 2012,Mon Mar 25 18:41:51 -0700 2013,Tue Mar 19 00:00:00 -0700 2013,NaN,0,0
478031,35cef391b171b4fca45771e508028212,10025305,8be463fed78f0da63e964706f710332b,things heating second novel devices . will beg...,Thu Jul 05 19:19:30 -0700 2012,Thu Jan 24 16:24:54 -0800 2013,Mon Jan 14 00:00:00 -0800 2013,NaN,0,0


In [19]:
# test the model with test dataset
res = model.predict([train['review_text'], train['n_comments'], train['n_votes']])

28125/28125 [==============================] - 80s 3ms/step


In [20]:
res

array([[-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.,  1.],
       ...,
       [-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.]], dtype=float32)

In [27]:
restest = model.predict([test['review_text'], test['n_comments'], test['n_votes']])

14939/14939 [==============================] - 36s 2ms/step


In [33]:
# reverse keras.utils.to_categorical for kaggle submission
ff = []
for line in tqdm(restest):
    tmp = -2
    category = None
    for i in (range(6)):
        if line[i] > tmp:
            category = i
            tmp = line[i]
    ff.append(category)
data = np.array(ff)

  0%|          | 0/478033 [00:00<?, ?it/s]

array([0, 0, 5, ..., 5, 0, 0])

In [39]:
test['rating'] = data

In [50]:
# create a dataframe for kaggle
id = test['review_id'].to_numpy()
rating = test['rating'].to_numpy()
df = pd.DataFrame( columns=['review_id', 'rating'])

In [51]:
df['review_id'] = id
df['rating'] = rating

In [54]:
# create a csv for submission
df.to_csv('submission_linear_model.csv',index=False )